In [1]:
import re
import glob
import os

import pandas as pd 

import pylint.lint
import nbformat

import sqlite3

In [42]:
! pip install nbformat 

Defaulting to user installation because normal site-packages is not writeable


# Data Directory Setup

In this study, data are split into 10 parts to handle them easily.</br> Use the directory pathonly if the data are gathered in a single directory.

In [3]:
# pylint_txt_path = ['data_directory']
pylint_txt_path = ['nb_data/part_0/KT_dataset/','nb_data/part_1/KT_dataset/','nb_data/part_2/KT_dataset/',
                   'nb_data/part_3/KT_dataset/','nb_data/part_4/KT_dataset/','nb_data/part_5/KT_dataset/',
                   'nb_data/part_6/KT_dataset/','nb_data/part_7/KT_dataset/','nb_data/part_8/KT_dataset/',
                   'nb_data/part_9/KT_dataset/'] 

output_path="result/all_pylint_nb_text/"

In [4]:
def get_list_of_filename(path, extension):
    notebook_filenames_list = glob.glob(os.path.join(path, extension))
    
    print(path, str(len(notebook_filenames_list)))
    return notebook_filenames_list

# Database SQlite Connection

## Database Function

In [5]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print( 'Connection established ', db_file)
        return conn
    
    except Error as e:
        print(e)

In [6]:
def close_connection(con):
    try:
        con.close()
        print ('Connection was closed now ', con)
    except Exception as e:
        print ('Connection invalid ', e)

In [7]:
def db_table_create(connection_obj):
    # Connecting to sqlite
    # connection object

    # cursor object
    cursor_obj = connection_obj.cursor()

    # Drop the table if already exists.
    cursor_obj.execute("DROP TABLE IF EXISTS NBISSUE")
    cursor_obj.execute("DROP TABLE IF EXISTS NBRATING")
    cursor_obj.execute("DROP TABLE IF EXISTS NBFILES")

    # Creating table
    table_nbissue = """ CREATE TABLE NBISSUE (
                Filename VARCHAR(5000) NOT NULL,
                CellNo INT,
                Line INT,
                Issue VARCHAR(10)
            ); """

    table_nbratings = """ CREATE TABLE NBRATING (
            Filename VARCHAR(5000) NOT NULL,
            Rating REAL
        ); """

    table_nbfiles = """ CREATE TABLE NBFILES (
        Filename VARCHAR(5000) NOT NULL,
        Type VARCHAR(10)
    ); """
    
    cursor_obj.execute(table_nbissue)
    cursor_obj.execute(table_nbratings)
    cursor_obj.execute(table_nbfiles)
    

    print("Table is Ready")
    connection_obj.commit()
    
    return cursor_obj

In [8]:
def data_insert_nbrating(connection, Filename, rating):
    cursor = connection.cursor()

    # Queries to INSERT records.    
    cursor.execute("insert into NBRATING (Filename, Rating) values (?, ?)",
            (Filename, rating))

    # Commit your changes in the database    
    connection.commit()

In [9]:
def data_insert_nbissue(connection, Filename, cell_no, line, issue):
    cursor = connection.cursor()

    # Queries to INSERT records.    
    cursor.execute("insert into NBISSUE (Filename, CellNo, Line, Issue) values (?, ?, ?, ?)",
            (Filename, cell_no, line, issue))

    # Commit your changes in the database    
    connection.commit()

In [10]:
def data_insert_nbfilenames(connection, Filename, mlnonmltype):
    cursor = connection.cursor()

    # Queries to INSERT records.    
    cursor.execute("insert into NBFILES (Filename, Type) values (?, ?)",
            (Filename, mlnonmltype))

    # Commit your changes in the database    
    connection.commit()

In [11]:
def display_data(conn, tablename):
    
    cursor = conn.cursor()
    # Display data inserted
    
    if (tablename == 'NBFILES'):
        print("Data Display for the table: NBFILES ")

        data=cursor.execute('''SELECT * FROM NBFILES''')
        for row in data:
            print(row)
        
    elif (tablename == 'NBRATING'):
        print("Data Display for the table: NBRATING ")

        data=cursor.execute('''SELECT * FROM NBRATING''')
        for row in data:
            print(row)
   
    elif   (tablename == 'NBISSUE'):  
        print("Data Display for the table: NBISSUE ")

        data=cursor.execute('''SELECT * FROM NBISSUE''')
        for row in data:
            print(row)
    
    conn.commit()

In [12]:
def count_data(conn, table):
    cursor = conn.cursor()
    
    rowsQuery = "SELECT Count() FROM %s" % table
    cursor.execute(rowsQuery)
    numberOfRows = cursor.fetchone()[0]
    print (table, numberOfRows)
    
    conn.commit()

In [13]:
def updateTable(connection):
    cursor = connection.cursor()
    query = "UPDATE NBISSUE SET FileID = FileID - 1"
    cursor.execute(query)
    
    # Commit your changes in the database    
    connection.commit()

## Database Execution 

In [14]:
connection = create_connection(r"result/all_pylint_nb_text/nb_pylint_result_with_type_v1.db")
# db_table_create(connection)


Connection established  result/all_pylint_nb_text/nb_pylint_result_with_type_v1.db


In [ ]:
# display_data(connection, 'NBRATING')
# display_data(connection, 'NBFILES')

count_data(connection, 'NBRATING')
# count_data(connection, 'NBFILES')

In [40]:
close_connection(connection)

Connection was closed now  <sqlite3.Connection object at 0x7fc2709466c0>


In [16]:
ml_filenames = pd.read_csv("result/ml_filename_v1.csv")
nonml_filenames = pd.read_csv("result/nonml_filename_v1.csv")
print(len(ml_filenames), len(nonml_filenames))
nonml_filenames['Filename']

171139 69156


0                     CVxTz_audio-data-augmentation.py
1        a01653716_hogar-ingresostrimestrales-enigh.py
2                   a03102030_eda-for-covid-19-data.py
3                      a03102030_eda-for-crime-data.py
4                 a03102030_eda-for-googleplay-data.py
                             ...                      
69151                zzy990106_single-roberta-large.py
69152                     zzy990106_truncate-errors.py
69153                 zzylordreturn_icu-requirement.py
69154                              zzzttt_newsubmit.py
69155                      zzzzzfy_kernel6464812e2f.py
Name: Filename, Length: 69156, dtype: object

In [17]:
filenames_for_issues, code_cells, line_numbers, issue_codes = [], [], [], []
filenames_for_rating, ratings = [], []
outputs_files = []


In [38]:
file_count = 19808

In [32]:
for nb_path in pylint_txt_path[6:]:
    nb_filenames = get_list_of_filename(nb_path, "*_pylintnb.txt")
    
    for file_name in nb_filenames:
        
    
#     data insert to nb filename and type (ml or nonml)
        nb_filename_for_type_check = file_name.split('/')[-1].split('_pylintnb.txt')[0]
        nb_filename_original = file_name.replace('_pylintnb.txt', '.ipynb')
        nb_type = 'ml'
        if (nonml_filenames[nonml_filenames['Filename'].str.contains(nb_filename_for_type_check, regex=True)].any(axis=None)):
            nb_type = 'nonml'
        data_insert_nbfilenames(connection, nb_filename_original, nb_type)
        # print (nb_filename_original)
            
        with open(file_name) as fp:
            lines_pylint = fp.readlines()
            
        for line in lines_pylint:
            try:
                if line.startswith('nb_data/'):
                    # print (line)
                    # filename = line.split(':')[0]
                    cell = line.split(':')[1].split('_')[1]
                    lineNo = line.split(':')[2]
                    issue = line.split(':')[4].strip()

                    data_insert_nbissue(connection, nb_filename_original, cell, lineNo, issue)
                    

                if line.startswith('Your code has been'):
                    rating = line.split(' ')[-1].split('/')[0]

                    data_insert_nbrating(connection, nb_filename_original, rating)


            except Exception as e:
                with open("exception_log.txt", "a") as file_object:
                    write_message = str(nb_filename_original) + " : " + str(e) + "\n"
                    file_object.write(write_message)

#         if (file_count % 1000 == 0):
#             with open("file_count_run.txt", "a") as file_object:
#                 file_object.write(nb_path+" "+file_count)
    

nb_data/part_6/KT_dataset/ 24876
nb_data/part_7/KT_dataset/ 24876
nb_data/part_8/KT_dataset/ 24876
nb_data/part_9/KT_dataset/ 24877


--- NB Analysis </br>
Total 248761

In [34]:
print ('done')

done


In [63]:
df_ratings_nb = pd.DataFrame({'Filename': filenames_for_rating, 'Rating': ratings})
df_ratings_nb

,Filename,Rating
0,nb_data/part_0/KT_dataset/basselali_simple.ipynb,1.67
1,nb_data/part_0/KT_dataset/efstathiasdrolia_wee...,8.25


In [64]:
df_issues_nb = pd.DataFrame({'Filename': filenames_for_issues, 'CellNo': code_cells, 'Line': line_numbers, 'Issue':issue_codes})
df_issues_nb

,Filename,CellNo,Line,Issue
0,nb_data/part_0/KT_dataset/basselali_simple.ipynb,1,3,C0303
1,nb_data/part_0/KT_dataset/basselali_simple.ipynb,1,9,C0301
2,nb_data/part_0/KT_dataset/basselali_simple.ipynb,1,0,C0114
3,nb_data/part_0/KT_dataset/basselali_simple.ipynb,1,11,C0411
4,nb_data/part_0/KT_dataset/basselali_simple.ipynb,1,5,W0611
5,nb_data/part_0/KT_dataset/efstathiasdrolia_wee...,1,2,C0303
6,nb_data/part_0/KT_dataset/efstathiasdrolia_wee...,1,0,C0114
7,nb_data/part_0/KT_dataset/efstathiasdrolia_wee...,1,0,C0103
8,nb_data/part_0/KT_dataset/efstathiasdrolia_wee...,3,1,W0104
9,nb_data/part_0/KT_dataset/efstathiasdrolia_wee...,8,1,W0104


In [47]:
nb = nbformat.read('nb_data/part_0/KT_dataset/basselali_simple.ipynb', as_version=4)
nb


{'cells': [{'cell_type': 'code',
   'execution_count': None,
   'metadata': {'collapsed': False},
   'outputs': [],
   'source': '# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load in \n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the "../input/" directory.\n# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory\n\nfrom subprocess import check_output\nprint(check_output(["ls", "../input"]).decode("utf8"))\nallData = pd.read_csv(\'../input/data.csv\')\nprint(allData.describe(include=\'all\'))\n# Any results you write to the current directory are saved as output.'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {'collapsed': False}

In [54]:
cell_counter = 1
for i in nb['cells']:
    if (i['cell_type'] == 'code'):
        line_counter = 1
        for line in i['source'].split('\n'):
            print(cell_counter, line)
    
    cell_counter += 1

1 # This Python 3 environment comes with many helpful analytics libraries installed
1 # It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
1 # For example, here's several helpful packages to load in 
1 
1 import numpy as np # linear algebra
1 import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
1 
1 # Input data files are available in the "../input/" directory.
1 # For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
1 
1 from subprocess import check_output
1 print(check_output(["ls", "../input"]).decode("utf8"))
1 allData = pd.read_csv('../input/data.csv')
1 print(allData.describe(include='all'))
1 # Any results you write to the current directory are saved as output.
2 
